# Welcome to VQ AutoEncoder clutstering tutorial!
ここでは、VQ (Vector Quantized) AutoEncoderによるでは、教師なしクラスタリング手法を行います。

### 目次

### 注意事項
このチュートリアルは演算負荷が高いためGPU環境が必要です。  
Google Colab で実行している方は、ページ上部から**ランタイム** &rarr; **ランタイムのタイプを変更** をクリックし **ハードウェアアクセラレータ** を *None* から *GPU* に変更してください。  
GPUが使用可能かどうかは次のコードブロックを実行することで分かります。

In [ ]:
import torch
print("GPU:",torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using:",device)

## ライブラリのインストールとインポート

In [ ]:
!pip install pytorch-lightning 
!pip install torchsummaryX 

In [ ]:
import torch
import torch.nn as nn 
import pytorch_lightning as pl
from dataclasses import dataclass
import numpy as np
import matplotlib.pyplot as plt 
from torchvision.utils import make_grid
from typing import *
from datetime import datetime
from torchsummaryX import summary
from torch.utils import data as dutil
from torchvision import transforms
from pytorch_lightning import loggers as pl_loggers
import os


## アルゴリズムの説明

#### VQ (Vector Quantization 日本語ではベクトル量子化) とは何か
この[サイト](https://aidiary.hatenablog.com/entry/20120813/1344853878)
より"VQ"とは、ベクトルで表されたデータ集合を有限個の代表的なパターン（セントロイド/量子化ベクトル）に置き換えることです。  
と、定義を言われても何かよくわかりませんよね。図を使って説明しましょう。

まずはデータ点があります。なんとなくまとまりがありそうですね。 
<br> 
<br> 
<img src="imgs/colab1.jpeg" width="500">

そしてデータ点の中からでもランダムでもいいのですが、適当に量子化ベクトル（セントロイド）を置きます。この量子化ベクトルの数を**量子化数**と呼びます。  
<br>
<img src="imgs/colab2.jpeg" width="500">

そして各データ点を、最も近い量子化ベクトルで置き換えます。これで量子化は完了しました。  
<br>
<img src="imgs/colab3.jpeg" width="500">


そしてデータ点に「一番目の量子化ベクトル」のようにラベルを割り振れば、クラスタリングとなります。  
データ点の分布に合わせてこの量子化ベクトルが適切な位置にあれば、うまくクラスタリングできていると言えますね！

#### VQ AutoEncoder clusteringとは
Encoderの出力を量子化ベクトルを使ってクラスタリングすることで、画像や音声などのデータに対し**教師なしで**ラベルを割り振ります。そのAutoEncoderの学習と、量子化ベクトルの学習（最適な位置に移動すること）を同時行うことが特徴です。

#### なぜVQ AE clusteringをするのか
ここまで読んできて、「k-means法で十分ではないか？」と疑問を持つ方もいるでしょう。この手法の目的は２つあります。  
1.  勾配降下法のみを用いてクラスタリングを行う。  
k-means法は「クラスターの平均値にセントロイドを移動させる」というアルゴリズムですから、深層学習のモデルに組み込む際に更新のアルゴリズムを分離する必要があります。VQAEでは全て勾配降下法により学習するためとても簡潔です。
2.  様々な「距離」の概念を使えるようにする。  
k-means法は基本的にユークリッド距離を用いることを前提にアルゴリズムが構築されています。しかし深層学習では多くの場面でコサイン類似度などの非ユークリッド距離を用います。VQ AEは「距離」という概念を直接使うため、k-means法では組み込みにくいデータパターンに対応できるのです。

## モデル（VQ AEの作り方）
VQ AutoEncoderを作るためには次のステップがあります。  
1. AutoEncoder(Encoder-Decoder) を作成する
2. 量子化ベクトルの数 x Encoderの出力次元数(Quantizing Dim) の重みを持つ`Quantizing Layer`を作る
3. `Encoder`, `Decoder`, `Quantizing Layer`を組み合わせて`VQ AutoEncoder`をつくる
4. 損失（誤差）関数を定義し、学習する  
<br>
さあ作っていきましょう。

### Encoder and Decoder
今回はMNIST手書き数字データセットを用いるので、入力は`BATCH x 1 x 28 x 28`になります。別の画像データにもモデルを対応させられるようにするために、ハイパーパラメータ`h`にその値を格納しています。  

`Encoder`は1x28x28の画像を 784 に平坦化します。その後は順にデータの次元数を小さくしています。  
> 1x28x28 &rarr; 784 &rarr; 256 &rarr; 128 &rarr; quantizing_dim

最後のレイヤーを`Tanh`にすることで、出力の範囲を -1 ~ 1 に納めています。


In [ ]:

class Encoder(nn.Module):
    def __init__(self,h):
        super().__init__()
        self.channels = h.channels
        self.width = h.width
        self.height = h.height
        self.input_size = (1,h.channels,h.width,h.height)
        self.output_size = (1,h.quantizing_dim)
        self.indim = h.channels * h.width* h.height
        self.h = h
        self.Eout_dim = h.quantizing_dim

        self.layers = nn.Sequential(
            nn.Flatten(1),
            nn.Linear(self.indim,256),nn.ReLU(),
            nn.Linear(256,128),nn.ReLU(),
            nn.Linear(128,self.Eout_dim),
            nn.Tanh(),
        )

    def forward(self,x:torch.Tensor):
        y = self.layers(x)
        return y
    
    def summary(self):
        dummy = torch.randn(self.input_size)
        summary(self,dummy)

DecoderはEncoderと逆のことをします。Encoderの出力を展開し、元々の画像データに復元します。  
> quantizing_dim &rarr; 128 &rarr; 256 &rarr; 784 &rarr; 1x28x28  

最後のレイヤーを`Sigmoid`にすることで、出力の範囲を 0 ~ 1 にしています。

In [ ]:
class Decoder(nn.Module):
    def __init__(self,h):
        super().__init__()
        self.channels = h.channels
        self.width = h.width
        self.height = h.height
        self.input_size = (1,h.quantizing_dim)
        self.output_size = (1,h.channels,h.width,h.height)
        self.h = h
        self.Dout_dim = self.channels*self.width*self.height

        self.layers = nn.Sequential(
            nn.Linear(h.quantizing_dim,128),nn.ReLU(),
            nn.Linear(128,256),nn.ReLU(),
            nn.Linear(256,self.Dout_dim),nn.Sigmoid(),
        )
    
    def forward(self,x:torch.Tensor):
        y = self.layers(x)
        y = y.view(-1,self.channels,self.height,self.width)
        return y

    def summary(self):
        dummy = torch.randn(self.input_size)
        summary(self,dummy)

### Quantizing layer
本日の主役、`Quantizing Layer (量子化層)`です！このレイヤーは`量子化ベクトルの個数 x Encoderの出力次元数`の学習可能な重みを持ちます。そして入力されたデータを最も近い量子化ベクトルに置換し、ついでに何番目の量子化ベクトルに置換されたかを表すidxも返します。このidxがクラスタリングされたラベルとなるのです！  
<img src="imgs/Qflow.jpeg" width="600">

　Quantizing Layerの重みの初期化時には、データと同じ分布に量子化ベクトルを用意する必要があります。なぜなら、量子化ベクトルを単純にランダムに取っても、データ点の分布と全く違う場所では量子化できないからです。  
　下のコードブロックでは、`initialized_by_dataset`という引数をTrueにすることで、データ点から直接重みを初期化することができます。もし、Quantizing Layerに入力されるデータの分布（Encoderの出力分布）が**正規分布と仮定されるのであれば**、`initialized_by_dataset=False`にしても大丈夫でしょう。

In [ ]:
class Quantizing(nn.Module):

    __initialized:bool = True

    def __init__(
        self, num_quantizing:int, quantizing_dim:int, _weight:torch.Tensor = None,
        initialize_by_dataset:bool = True, mean:float = 0.0, std:float = 1.0,
        dtype:torch.dtype = None, device:torch.device = None
        ):
        super().__init__()
        assert num_quantizing > 0
        assert quantizing_dim > 0
        self.num_quantizing = num_quantizing
        self.quantizing_dim = quantizing_dim
        self.initialize_by_dataset = initialize_by_dataset
        self.mean,self.std = mean,std

        if _weight is None:
            self.weight = nn.Parameter(
                torch.empty(num_quantizing, quantizing_dim ,dtype=dtype,device=device)
            )
            nn.init.normal_(self.weight, mean=mean, std=std)

            if initialize_by_dataset:
                self.__initialized = False
                self.__initialized_length= 0

        elif type(_weight) is torch.Tensor:
            assert _weight.dim() == 2
            assert _weight.size(0) == num_quantizing
            assert _weight.size(1) == quantizing_dim
            self.weight = nn.Parameter(_weight.to(device).to(dtype))

        else:
            raise ValueError("Weight type is unknown type! {}".format(type(_weight)))

    def forward(self,x:torch.Tensor) -> Tuple[torch.Tensor]:
        """
        x   : shape is (*, E), and weight shape is (Q, E). 
        return -> ( quantized : shape is (*, E), quantized_idx : shape is (*,) )
        """
        input_size = x.shape
        h = x.view(-1,self.quantizing_dim) # shape is (B,E)
        
        self.initialize_weight(x)
        
        delta = self.weight.unsqueeze(0) - h.unsqueeze(1) # shape is (B, Q, E)
        dist =torch.sum(delta*delta, dim=-1) # shape is (B, Q)
        q_idx = torch.argmin(dist,dim=-1) # shape is (B,)
        q_data = self.weight[q_idx] # shape is (B, E)

        return q_data.view(input_size), q_idx.view(input_size[:1])
    
    def initialize_weight(self, x:torch.Tensor):
        h = x.view(-1,self.quantizing_dim) # shape is (B,E)
        if not self.__initialized and self.initialize_by_dataset:
            getting_len = self.num_quantizing - self.__initialized_length
            init_weight = h[torch.randperm(len(h))[:getting_len]]
            
            _until = self.__initialized_length + init_weight.size(0)
            self.weight.data[self.__initialized_length:_until] = init_weight
            self.__initialized_length = _until
            print('replaced weight')

            if _until >= self.num_quantizing:
                self.__initialized = True
                print('initialized')
    
    @property
    def is_initialized(self):
        return self.__initialized

    @is_initialized.setter
    def is_initialized(self, b:bool):
        self.__initialized = b
        if b:
            self.__initialized_length= num_quantizings
        else:
            self.__initialized_length = 0
    



### VQ AutoEncoder
もはや材料は揃いました。VQ AutoEncoderを構築していきましょう。今回作るモデルの概要はこちらです。
<img src="imgs/vqae.png" width="600">

Encoderの出力である$E_{out}$はQuantizing Layerに入力され、最も近い量子化ベクトルである$Q_{out}$になり出力されます。(ついでに量子化ベクトルのインデックスも!)  
Decoderはいつも通り$E_{out}$を受け取り再構成して画像を出力します。

#### 損失（誤差）関数の取り方
今回の損失は`reconstruction_loss(再構成誤差)` と `quatizing_loss(量子化誤差)`の和を使用します。  
`再構成誤差`はいつも通りAutoEncoderの入力画像と出力画像のMSE(平均二乗誤差）を使い、それを最小化します。 
`量子化誤差`は、$E_{out}$と$Q_{out}$の*距離*を使い、それを最小化します。$E_{out}$は変化せず、$Q_{out}$の方が学習されます。（Encoderに対してはbackpropagation(誤差逆伝搬)を行わない)  
<br>
今回`量子化誤差`は`Quantizing Layer`でユークリッド距離を使用しているため、それに近い距離の概念であるMSE(平均二乗誤差)を使用します。
$Q_{out}$が$E_{out}$と全く同じなりそうですが、実際一つの量子化ベクトルは複数のデータ点と*距離*を最小化しようとするため、だいたい *距離* が平均的に最小になる位置に量子化ベクトルは移動します。

#### 記録について
記録するものはたくさんあります。全てTensorboardに記録されます。
- 学習時のハイパーパラメータ  
  学習率(learning rate)などを記録します。  
  <br>
- 再構成誤差(reconstruction_loss/r_loss)  
  これは純粋にAutoEncoderの損失です。  
  <br>
- 量子化誤差(quantizing_loss/q_loss)   
  量子化ベクトルとEncoderの出力との平均距離です。  
  <br>
- 損失(loss)  
  loss = r_loss + q_loss  
  <br>
- 量子化再構成誤差(reconstructed_quantizing_loss/rq_loss)  
  これは量子化層の出力をDecoderに通し、再構成した画像とEncoderの入力画像との誤差です。  
  <br>
- 画像の記録 
  下のような形で記録されます。上から、入力画像、再構成画像、量子化後($Q_{out}$)の再構成画像となっています。 
  <br>
  <img src="imgs/logimg.png" height="100">  
  <br>
- １つの量子化ベクトルごとの割り当て数
  下のような形で記録されます。理想的には一様分布になります。
  <br>
  <img src="imgs/histogram.png" width="300">
  <br>

  

次のコードブロックでは、レイヤーの定義、重みの初期化、学習、記録の処理を全て記述しています。

In [ ]:

class VQ_AutoEncoder(pl.LightningModule):

    def __init__(self,h):
        super().__init__()
        self.model_name = h.model_name
        self.h = h
        self.num_quantizing = h.num_quantizing
        self.quantizing_dim = h.quantizing_dim
        self.lr = h.lr
        self.my_hparams_dict = h.get()

        # set criterion
        self.reconstruction_loss = nn.MSELoss()
        self.quantizing_loss = nn.MSELoss()
        
        # set histogram
        self.q_hist = torch.zeros(self.num_quantizing,dtype=torch.int)
        self.q_hist_idx = np.arange(self.num_quantizing)
        # set layers
        self.encoder = Encoder(h)
        self.quantizer = Quantizing(h.num_quantizing,h.quantizing_dim)
        self.decoder = Decoder(h)

        self.input_size = self.encoder.input_size
        self.output_size = self.input_size

    def forward(self,x:torch.Tensor):
        h = self.encoder(x)
        Qout,Qidx = self.quantizer(h)
        y = self.decoder(Qout)
        return y

    def configure_optimizers(self):
        optim = torch.optim.Adam(self.parameters(),self.lr)
        return optim
    
    @torch.no_grad()
    def set_quantizing_weight(self,data_loader,device='cpu'):
        self.quantizer.is_initialized = False
        for batch in data_loader:
            data,_ = batch
            data = data.to(device)
            Eout = self.encoder(data)
            self.quantizer.initialize_weight(Eout)
            if self.quantizer.is_initialized:
                break

        torch.cuda.empty_cache()

    def on_fit_start(self) -> None:
        self.logger.log_hyperparams(self.my_hparams_dict)

    def training_step(self,batch,idx):
        data,_  = batch
        self.view_data = data
        Eout = self.encoder(data)
        Qtgt = Eout.detach()
        Qout,Qidx = self.quantizer(Qtgt)
        out = self.decoder(Eout)

        # loss
        r_loss = self.reconstruction_loss(out,data)
        q_loss = self.quantizing_loss(Qout,Qtgt)
        loss = r_loss + q_loss

        # log
        rq_loss = self.reconstruction_loss(self.decoder(Qout),data)
        self.log('loss',loss)
        self.log('reconstruction_loss',r_loss)
        self.log('quantizing_loss',q_loss)
        self.log('reconstructed_quantizing_loss',rq_loss)

        idx,count = torch.unique(Qidx,return_counts = True)
        self.q_hist[idx.cpu()] += count.cpu()
        return loss

    @torch.no_grad()
    def on_epoch_end(self) -> None:
        if (self.current_epoch+1) % self.h.view_interval ==0:
            # image logging
            data = self.view_data[:self.h.max_view_imgs].float()
            data_len = len(data)
            Eout = self.encoder(data)
            Qout,Qidx = self.quantizer(Eout)
            out = self.decoder(Eout)
            Qdecoded = self.decoder(Qout)

            grid_img = make_grid(torch.cat([data,out,Qdecoded],dim=0),nrow=data_len)
            self.logger.experiment.add_image("MNIST Quantizings",grid_img,self.current_epoch)

            # histogram logging
            fig = plt.figure(figsize=(6.4,4.8))
            ax = fig.subplots()
            ax.bar(self.q_hist_idx,self.q_hist)
            
            quantized_num = len(self.q_hist[self.q_hist!=0])
            q_text = f'{quantized_num}/{self.num_quantizing}'
            ax.text(0.9,1.05,q_text,ha='center',va='center',transform=ax.transAxes,fontsize=12)
            ax.set_xlabel('weight index')
            ax.set_ylabel('num')
            self.logger.experiment.add_figure('Quantizing Histogram',fig,self.current_epoch)
            
        self.q_hist.zero_()

    def summary(self,tensorboard=False):
        from torch.utils.tensorboard import SummaryWriter
        dummy = torch.randn(self.input_size)
        summary(self,dummy)

        if tensorboard:
            writer = SummaryWriter(comment=self.model_name,log_dir="VQAE_log")
            writer.add_graph(self,dummy)
            writer.close()


学習の処理を簡単に書くために、`pytorch-lightning`というライブラリを使用しています。  
`pytorch-lightning`では下のような形でモデルを定義することができます。  
```python
class ModelName(pl.LightningModule):

    def __init__(self, arguments): # required
        super().__init__()
        ################################################
        # この中でレイヤーや、criterion(Loss関数)を定義します。
        self.criterion = SomeLossFunc()
        self.layer = SomeLayers()
        ################################################

    def forward(self, input): # not required (when training)
        ################################################
        # この中にデータの流れを書きますが、学習する際にこの
        # forward methodは使われません。
        output = self.layer(input)
        ################################################
        return output

    def configure_optimizers(self): # required
        ################################################
        # この中でoptimizerを定義して、returnで返します。
        optim = torch.optim.SomeOptimizer(self.parameters(), lr=lr)
        ################################################
        return optim
    
    def training_step(self, batch, idx): # required
        ################################################
        # この中で学習する時のデータの流れを書きます。学習するとき
        # は"損失"まで計算し、それを return で返します。学習時に
        # 記録したい値は self.log("name", value) で記録するこ
        # とができます。
        input, answer = batch # extracting
        output = self.layer(input) # flow
        loss = self.criterion(output, answer) # calculate loss
        self.log("loss",loss) # log
        return loss # return (required)
        ################################################

    def validation_step(self, batch, idx): # not required when you don't need the validation.
        ################################################
        # この中に検証用データセットでの処理を書きます。この関数で
        # は値をreturnしなくて良いです。self.log("name", value)
        # で値を記録してください。
        # Trainerにvalidation用のDataLoaderを与えなかった場合、
        # この関数はつかわれません。　
        ################################################
```

Event drivenなので、他にも様々な関数が用意されています。pytorch_lightningの`Trainer`が自動的にそのタイミングでオーバーライドされた関数を呼び出します。
```python
model = ModelName(arguments)
trainer = pl.Trainer(gpus=1,precsion=16, max_epochs=10,...)
trainer.fit(model,train_loader, validation_loader)
```

### hyper parameter の定義

In [ ]:
@dataclass
class hparam:
    model_name:str = "VQ_AutoEncoder"
    max_view_imgs = 16
    view_interval = 10

    lr:float = 0.001

    num_quantizing:int = 32
    quantizing_dim:int = 32

    channels:int = 1
    width:int = 28
    height:int = 28

    def get(self):
        return self.__dict__

## 学習する

 ### Tensor Board
 Google Colabでこのノートブックを実行している方は、次のコードブロックを実行してください。  
 ローカルマシンで実行している方はノートブックと同じディレクトリでターミナルを開き、次のコマンドを実行してください。    
 `tensorboard --logdir="VQAE_log"`

In [ ]:
%load_ext tensorboard
%tensorboard --logdir="VQAE_log"

### データセットのロード(MNIST)

In [ ]:
from torchvision.datasets import MNIST
dataset = MNIST(
    "data",train=False,download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
    ])
)

### 時刻を取得する関数

In [ ]:
def get_now(strf:str = '%Y-%m-%d_%H-%M-%S'):
    now = datetime.now().strftime(strf)
    return now

### パラメータの保存について

In [ ]:
def param_dir():
    if not os.path.exists("params"):
        os.makedirs("params")
        
def save_params(model:VQ_AutoEncoder, now):
    param_dir()
    torch.save(model.encoder.state_dict(),f"params/{model.model_name}_{now}.encoder.pth")
    torch.save(model.decoder.state_dict(),f"params/{model.model_name}_{now}.decoder.pth")
    torch.save(model.quantizer.state_dict(),f"params/{model.model_name}_{now}.quantizing.pth")
    torch.save(model.state_dict(),f"params/{model.model_name}_{now}.pth")

### データローダーの定義

In [ ]:
BATCH_SIZE = 1024
dataloader = dutil.DataLoader(
    dataset, BATCH_SIZE,shuffle=True, num_workers=0, pin_memory=True, drop_last=True
)

### pytorch lightning の key words


In [ ]:
pl_kwds = {
    "gpus": 1 if torch.cuda.is_available() else 0,
    "precision": 16,
    "max_epochs": 500,
    "log_every_n_steps":5,
}

### 学習（1回目）
一回目の学習をしていきましょう。ネタバレですが、うまく量子化できません。ですが学習後のEncoderとDecoderの重みを使用するため必ず実行してください。

##### インスタンス

In [ ]:
h = hparam(model_name="VQAE_pure")
model = VQ_AutoEncoder(h)
model.summary(True)

##### Quantinzing Weight をセット

In [ ]:
model.set_quantizing_weight(dataloader)

##### 実行

In [ ]:
logger = pl_loggers.TensorBoardLogger("VQAE_log/pure")
trainer = pl.Trainer(logger=logger, **pl_kwds)
trainer.fit(model, dataloader)
logger.close()
now = get_now()
print(now)

##### パラメータ保存

In [ ]:
save_params(model,now)

##### なぜうまくいかないのか
このようなヒストグラムは確認できましたか？  
結果をTensorboardで確認するとわかりますが、全くクラスタリングできていません。全てのデータがただ一つの量子化ベクトルに割当たってしまっています。  
この原因は学習初期のEncoderの出力分布が大きく変化するためでしょう。量子化ベクトルがEncoderの出力$E_{out}$の変化を追いきれないのです。
<br>
<img src="imgs/failed.png" width="400">

### 学習（通常）
ということで、ほとんど学習が終わったEncoderとDecoderの重みを使いましょう。

##### パラメータのロード

In [ ]:
def load_trained_param(model:VQ_AutoEncoder):
    model.encoder.load_state_dict(torch.load("params/~.encoder.pth"))    
    model.decoder.load_state_dict(torch.load("params/~.decoder.pth"))

##### インスタンスと実行

In [ ]:
def train(settings:Dict[str,Any] = dict(), pl_kwds:Dict[str, Any]=dict()):
    h = hparam(**settings)
    model = VQ_AutoEncoder(h)
    load_trained_param(model)
    model.set_quantizing_weight(dataloader)
    logger = pl_loggers.TensorBoardLogger("VQAE_log/fromTraining")
    trainer = pl.Trainer(logger=logger, **pl_kwds)
    trainer.fit(model, dataloader)
    now = get_now()
    logger.close()
    save_params(model, now)

In [ ]:
pl_kwds = {
    "gpus": 1 if torch.cuda.is_available() else 0,
    "precision": 16,
    "max_epochs": 100,
    "log_every_n_steps":5,
}

In [ ]:
train(pl_kwds=pl_kwds)

##### Ex) changing `num_quantizing`
量子化数を変化させて結果がどのように変わるか見てみましょう。

In [ ]:
num_quantizings = [8,128,512]
for nq in num_quantizings:
    setting = {
        "model_name":"vqae_changing_num_quantizing_{}".format(nq),
        "num_quantizing":nq,
    }
    train(setting, pl_kwds)

### 結果を解析する

### 考えること